In [135]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [136]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, f1_score, accuracy_score
from matplotlib import pyplot as plt
from sklearn import tree
from sklearn.ensemble import HistGradientBoostingClassifier

In [137]:
path = 'train.csv'
data = pd.read_csv(path)

pathTest = 'test_nolabel.csv'
dataTest = pd.read_csv(pathTest)


In [138]:

def transform_day(day):
    if day == "mon":
        return 0
    elif day == "tue":
        return 1
    elif day == "wed":
        return 2
    elif day == "thu":
        return 3
    elif day == "fri":
        return 4
    return None

def transform_month(month):
    if month == "jan":
        return 0
    elif month == "feb":
        return 1
    elif month == "mar":
        return 2
    elif month == "apr":
        return 3
    elif month == "may":
        return 4
    elif month == "jun":
        return 5
    elif month == "jul":
        return 6
    elif month == "aug":
        return 7
    elif month == "sep":
        return 8
    elif month == "oct":
        return 9
    elif month == "nov":
        return 10
    elif month == "dec":
        return 11
    return None

def transform_loan(loan):
    if loan == "yes":
        return 1
    elif loan == "no":
        return 0
    return None


def transform_default(default):
    if default == "yes":
        return 1
    elif default == "no":
        return 0 
    return None


def transform_housing(housing):
    if housing == "yes":
        return 1
    elif housing == "no":
        return 0
    return None

Para el preprocesado utilizamos varias formas.
Primero utilizamos pandas.get_dummies para codificar las variables categóricas mediante one-hot encoding. Así como funciones propias para otras columnas.

In [139]:
#Preprocesado
data= data.drop(['default'],axis=1)


#Dummies
dummies = pd.get_dummies(data['marital'])
dummies = dummies.drop(labels='unknown', axis=1)
data = pd.concat([data,dummies], axis=1)
data = data.drop(['marital'], axis=1)

dummies2 = pd.get_dummies(data['job'])
dummies2 = dummies2.drop(labels='unknown', axis=1)
data = pd.concat([data,dummies2], axis=1)
data = data.drop(['job'], axis=1)

dummies3 = pd.get_dummies(data['education'])
dummies3 = dummies3.drop(labels='unknown', axis=1)
data = pd.concat([data,dummies3], axis=1)
data = data.drop(['education'], axis=1)

dummies4 = pd.get_dummies(data['contact'])
data = pd.concat([data,dummies4], axis=1)
data = data.drop(['contact'], axis=1)

dummies5 = pd.get_dummies(data['poutcome'])
data = pd.concat([data,dummies5], axis=1)
data = data.drop(['poutcome'], axis=1)

#Funciones propias
data['loan'] = data['loan'].apply(transform_loan)
#data['default'] = data['default'].apply(transform_default)
data['housing'] = data['housing'].apply(transform_housing)
data['day_of_week'] = data['day_of_week'].apply(transform_day)
data['month'] = data['month'].apply(transform_month)


#data = data.dropna()



In [140]:
#Preprocesado
dataTest= dataTest.drop(['default'],axis=1)


#Dummies
dummies = pd.get_dummies(dataTest['marital'])
dummies = dummies.drop(labels='unknown', axis=1)
dataTest = pd.concat([dataTest,dummies], axis=1)
dataTest = dataTest.drop(['marital'], axis=1)

dummies2 = pd.get_dummies(dataTest['job'])
dummies2 = dummies2.drop(labels='unknown', axis=1)
dataTest = pd.concat([dataTest,dummies2], axis=1)
dataTest = dataTest.drop(['job'], axis=1)

dummies3 = pd.get_dummies(dataTest['education'])
dummies3 = dummies3.drop(labels='unknown', axis=1)
dataTest = pd.concat([dataTest,dummies3], axis=1)
dataTest = dataTest.drop(['education'], axis=1)

dummies4 = pd.get_dummies(dataTest['contact'])
dataTest = pd.concat([dataTest,dummies4], axis=1)
dataTest = dataTest.drop(['contact'], axis=1)

dummies5 = pd.get_dummies(dataTest['poutcome'])
dataTest = pd.concat([dataTest,dummies5], axis=1)
dataTest = dataTest.drop(['poutcome'], axis=1)

#Funciones propias
dataTest['loan'] = dataTest['loan'].apply(transform_loan)
#dataTest['default'] = dataTest['default'].apply(transform_default)
dataTest['housing'] = dataTest['housing'].apply(transform_housing)
dataTest['day_of_week'] = dataTest['day_of_week'].apply(transform_day)
dataTest['month'] = dataTest['month'].apply(transform_month)


#dataTest = dataTest.dropna()

In [141]:
data

,Id,age,housing,loan,month,day_of_week,duration,campaign,pdays,previous,...,basic.9y,high.school,illiterate,professional.course,university.degree,cellular,telephone,failure,nonexistent,success
0,44dc8df9,35,0.0,0.0,6,4,189,1,999,0,...,0,1,0,0,0,0,1,0,1,0
1,3f62aa06,52,0.0,0.0,5,4,188,1,999,0,...,1,0,0,0,0,0,1,0,1,0
2,34bd6148,44,0.0,0.0,4,0,267,4,999,1,...,1,0,0,0,0,1,0,1,0,0
3,e1671cba,35,1.0,1.0,5,4,14,9,999,0,...,0,1,0,0,0,0,1,0,1,0
4,c20477f0,33,1.0,0.0,6,3,52,7,999,0,...,0,0,0,0,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36183,a97228fc,58,1.0,0.0,6,3,214,4,999,2,...,0,1,0,0,0,1,0,1,0,0
36184,e68d72d1,30,1.0,0.0,3,3,243,3,999,0,...,0,0,0,0,1,1,0,0,1,0
36185,70346140,41,0.0,0.0,3,4,135,1,999,1,...,1,0,0,0,0,1,0,1,0,0
36186,b3b92241,29,1.0,0.0,6,3,480,1,6,1,...,0,1,0,0,0,1,0,0,0,1


In [142]:
features = data.columns[1:-1]

X = data[features].values            
y = data['Expected'].values
print(y)
print(data.Expected)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

[0 0 0 ... 0 1 0]
0        0
1        0
2        0
3        0
4        0
        ..
36183    0
36184    1
36185    0
36186    1
36187    0
Name: Expected, Length: 36188, dtype: int64


In [143]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() 
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [144]:

parameters = {
    "histGradientBoosting": {
        "loss": ["log_loss", "auto"], # two possible values for the criterion
        "max_depth": [2, 5, 10], # possible values for the max depth of the tree model
    }}

gs1 = GridSearchCV(HistGradientBoostingClassifier(), param_grid=parameters["histGradientBoosting"], cv=5, n_jobs=1)
gs1.fit(X_train, y_train)
print("Best params are:", gs1.best_params_)
print("Best score is:", gs1.best_score_)


C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\ensemble\_hist_gradient_boosting\gradient_boosting.py:1823: FutureWarning: The loss 'auto' was deprecated in v1.1 and will be removed in version 1.3. Use 'log_loss' which is equivalent.
  warnings.warn(
C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\ensemble\_hist_gradient_boosting\gradient_boosting.py:1823: FutureWarning: The loss 'auto' was deprecated in v1.1 and will be removed in version 1.3. Use 'log_loss' which is equivalent.
  warnings.warn(
C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\ensemble\_hist_gradient_boosting\gradient_boosting.py:1823: FutureWarning: The loss 'auto' was deprecated in v1.1 and will be removed in version 1.3. Use

Best params are: {'loss': 'log_loss', 'max_depth': 2}
Best score is: 1.0


In [145]:
dataTest.shape

(5000, 41)

In [147]:
features = dataTest.columns[1:-1]
X2 = dataTest[features].values
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y, test_size=0.25, random_state=0) 

model = HistGradientBoostingClassifier(loss='log_loss',max_depth=2)

predictions1 = gs1.best_estimator_.predict(X_train)

print(classification_report(y_train, predictions1))
print("F-score", f1_score(y_train, predictions1))


model.fit(X_train, y_train)
prediccion = model.predict(X2_test)
print("F-score test", f1_score(y_test, prediccion))
print("La prediccion es: ",prediccion)
plt.figure(figsize=(10,10))
tree.plot_tree(model)
plt.show()

ValueError: Found input variables with inconsistent numbers of samples: [5000, 36188]

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
#adaboost con decision tree
clf = AdaBoostClassifier(n_estimators=100, random_state=0, learning_rate=1)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print("F-score test", f1_score(y_test, predictions))
print("La prediccion es: ",predictions)

F-score test 1.0
La prediccion es:  [0 0 1 ... 0 0 0]


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
#Support vector machine para adaboost
svc=SVC(probability=True, kernel='linear')
#tarda mucho en correr
clf = AdaBoostClassifier(n_estimators=3, random_state=0, base_estimator=svc, learning_rate=1)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print("F-score test", f1_score(y_test, predictions))
print("La prediccion es: ",predictions)

F-score test 1.0
La prediccion es:  [0 0 1 ... 0 0 0]


In [17]:
data.head(5000)

,Id,age,housing,loan,month,day_of_week,duration,campaign,pdays,previous,...,basic.9y,high.school,illiterate,professional.course,university.degree,cellular,telephone,failure,nonexistent,success
Id,,,,,,,,,,,,,,,,,,,,,
44dc8df9,44dc8df9,35,0.0,0.0,6,4,189,1,999,0,...,0,1,0,0,0,0,1,0,1,0
3f62aa06,3f62aa06,52,0.0,0.0,5,4,188,1,999,0,...,1,0,0,0,0,0,1,0,1,0
34bd6148,34bd6148,44,0.0,0.0,4,0,267,4,999,1,...,1,0,0,0,0,1,0,1,0,0
e1671cba,e1671cba,35,1.0,1.0,5,4,14,9,999,0,...,0,1,0,0,0,0,1,0,1,0
c20477f0,c20477f0,33,1.0,0.0,6,3,52,7,999,0,...,0,0,0,0,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
f51b1019,f51b1019,52,1.0,0.0,4,4,412,1,999,0,...,0,0,0,0,1,0,1,0,1,0
bfc4e380,bfc4e380,36,0.0,0.0,4,0,144,7,999,0,...,0,1,0,0,0,0,1,0,1,0
e0d746df,e0d746df,61,1.0,0.0,5,2,1076,2,999,0,...,0,0,0,0,1,1,0,0,1,0
